In [1]:
import pandas as pd

In [2]:
z031 = pd.read_csv("zindex031_iconsense_extractie_20230825.csv", sep = ';', low_memory=False) #Handelsproducten
z701 = pd.read_csv("zindex701_iconsense_extractie_20230825.csv", sep=';', low_memory=False) # Samenstelling (HPK's per ingrediënt)
z750 = pd.read_csv("zindex750_HiX_extractie_20230825.csv", sep=',', low_memory=False) # Generieke namen
z051 = pd.read_csv("zindex051_iconsense_extractie_20230829.csv", sep=';', low_memory=False) # Voorschrijfproducten
z711 = pd.read_csv("zindex711_iconsense_extractie_20230829.csv", sep=';', low_memory=False) # Generieke producten
z902 = pd.read_csv("zindex902_iconsense_extractie_20230829.csv", sep=';', low_memory=False) # Thesauri algemeen
z801 = pd.read_csv("zindex801_iconsense_extractie_20230829.csv", sep=';', low_memory=False) # ATC codes

In [3]:
relevant_columns_750 = ['Generiekenaamcode','Generieke naam']
relevant_columns_701 = ['HPKODE', 'GNMWHS', 'GNGNK', 'GNMINH'] #GNGNK = Generiekenaamcode
relevant_columns_031 = ['HPKODE', 'NMNAAM', 'PRKODE'] #https://www.z-index.nl/documentatie/bestandsbeschrijvingen/bestand?bestandsnaam=BST031T
relevant_columns_051 = ['PRKODE', 'GPKODE'] #https://www.z-index.nl/documentatie/bestandsbeschrijvingen/bestand?bestandsnaam=BST051T
relevant_columns_711 = ['THKTWG','GPKODE', 'GPKTWG','ATCODE'] #https://www.z-index.nl/documentatie/bestandsbeschrijvingen/bestand?bestandsnaam=BST711T
relevant_columns_902 = ['TSNR','TSITNR', 'THNM50'] #https://www.z-index.nl/documentatie/bestandsbeschrijvingen/bestand?bestandsnaam=BST711T
relevant_columns_801 = ['ATCODE', 'ATOMS'] #https://www.z-index.nl/documentatie/bestandsbeschrijvingen/bestand?bestandsnaam=BST801T

## Filter thesaurus on thesarus '7' (=toedienwegen)
z902_f = z902.loc[z902['TSNR'] == 7]

##remove trailing leading zero's
z902_f['TSITNR'] = z902_f['TSITNR'].apply(lambda x: str(x).lstrip('0'))

## format to string without decimal
z711['GPKTWG'] = z711['GPKTWG'].astype('Int64').astype(str)

KM_t01a =z701[relevant_columns_701].merge(z031[relevant_columns_031], how= 'left', on= 'HPKODE'
                                            ).merge(z750[relevant_columns_750], how= 'left', left_on= 'GNGNK', right_on='Generiekenaamcode'
                                                    ).merge(z051[relevant_columns_051], how='left', on='PRKODE'
                                                            ).merge(z711[relevant_columns_711], how='left', on='GPKODE'
                                                                    ).merge(z902_f[relevant_columns_902], how='left', left_on='GPKTWG', right_on='TSITNR'
                                                                            ).merge(z801[relevant_columns_801], how='left', on='ATCODE')

/tmp/ipykernel_17618/3816285658.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z902_f['TSITNR'] = z902_f['TSITNR'].apply(lambda x: str(x).lstrip('0'))


In [4]:
# Step 1: Ensure 'Generieke naam' is of string type and handle missing values
KM_t01a['Generieke naam'] = KM_t01a['Generieke naam'].astype(str).fillna('')

# Step 1b: Remove all rows with active component
KM_t01a_excipients_only = KM_t01a.loc[KM_t01a['GNMWHS'] == 'H']

# Step 2: Group by 'NMNAAM' and aggregate excipients
KM_grouped = KM_t01a_excipients_only.groupby('NMNAAM').agg({
    # Aggregate excipients by joining unique values with semicolon
    'Generieke naam': lambda x: ';'.join(x.dropna().unique()),
    
    # For other columns, decide on the aggregation method
    # If these columns have the same value per 'NMNAAM', you can take the first occurrence
    'HPKODE': 'first',
    'GNMWHS': 'first',
    'GNGNK': 'first',
    'GNMINH': 'first',
    'PRKODE': 'first',
    'Generiekenaamcode': 'first',
    'GPKODE': 'first',
    'THKTWG': 'first',
    'GPKTWG': 'first',
    'ATCODE': 'first',
    'TSNR': 'first',
    'TSITNR': 'first',
    'THNM50': 'first',
    'ATOMS': 'first'
}).reset_index()

In [5]:
# Load excipient status CSV
excipient_status_df = pd.read_csv('hulpstof_beoordelingsbestand_KetoMed_v20241230.csv', encoding=' latin1',sep=';')

# Create mapping dictionary (ensure uppercase for consistency)
excipient_status_map = dict(zip(excipient_status_df['Excipient'].str.upper(), excipient_status_df['Conclusion Ketoproof'].str.upper()))

def determine_status_and_problematic_excipients(excipients, mapping):
    if pd.isna(excipients) or excipients.strip() == '':
        return pd.Series(['Unknown', '', ''])
    
    # Split the excipients by semicolon and strip whitespace
    excipient_list = [exc.strip().upper() for exc in excipients.split(';')]
    
    has_no = False
    has_unknown = False
    non_ketoproof_excipients = []
    unknown_excipients = []
    
    for exc in excipient_list:
        status = mapping.get(exc, 'UNKNOWN')  # Default to 'UNKNOWN' if not found
        if status == 'NO':
            has_no = True
            non_ketoproof_excipients.append(exc.title())
        elif status == 'UNKNOWN':
            has_unknown = True
            unknown_excipients.append(exc.title())
    
    # Determine Handelsproduct Status
    if has_no:
        status = 'No'
    elif has_unknown:
        status = 'Unknown'
    else:
        status = 'Yes'
    
    # Create semicolon-separated strings
    non_ketoproof_str = '; '.join(non_ketoproof_excipients) if non_ketoproof_excipients else ''
    unknown_excipients_str = '; '.join(unknown_excipients) if unknown_excipients else ''
    
    return pd.Series([status, non_ketoproof_str, unknown_excipients_str])

# Apply the function and expand the returned Series into three new columns
KM_grouped[['Handelsproduct Status', 'Non-Ketoproof Excipients', 'Unknown Excipients']] = KM_grouped['Generieke naam'].apply(
    lambda x: determine_status_and_problematic_excipients(x, excipient_status_map)
)


In [6]:
KM_grouped.to_csv('labeled_drugs.csv', encoding='latin1', index=False)